# 二分匹配问题

## 什么是二分图？

二分图是一种特殊的无向图，图中所有顶点可以被划分到两个不相交的独立集合 $U$ 和 $V$ 中，且图所有的边都是一端在集合 $U$ 另一端在集合 $V$ 中。

+ 二分图的严格定义：
> 无向图G为二分图的充分必要条件是，G至少有两个顶点，且其所有回路的长度均为偶数。如果无回路，相当于任一回路的次数为0，0也是偶数，故也视为二分图。

## 二分图的最大匹配

对于边上不带权重的二分图，找到拓扑图中边的数量最多的匹配边集，且匹配边集中任意边之间没有公共顶点。**匹配边集中的边被称为匹配边，匹配边两端的顶点被称为匹配点，反之则称为未匹配边和未匹配点。**

## 二分图的完美匹配

如果一个二分图的最大匹配中的边的数目等于图中顶点数目的一半，那个这个匹配就叫做该二分图的完美匹配。

## 二分图的最大完美匹配

对于边上带有权重值的二分图，所有匹配边上权重值之和最大的完美匹配，就叫做二分图的最大完美匹配。


In [ ]:
import algviz

graph_nodes = [
    [0, 0], [1, 1], [2, 2], [3, 3],
    [4, 4], [5, 5], [6, 6], [7, 7],
]
graph_edges = [
    [0, 4], [0, 5], [1, 4], [1, 6],
    [2, 5], [3, 5], [2, 7]
]

# 构建拓扑图
graph_map = algviz.parseGraph(graph_edges, graph_nodes, directed=False)
viz = algviz.Visualizer()
graph = viz.createGraph(list(graph_map.values()), directed=False, horizontal=True)

# 二分图左侧和右侧节点
left_nodes = [0, 1, 2, 3]
right_nodes = [4, 5, 6, 7]

viz.display(1)

# 匈牙利算法

## 增广路径的定义

**从二分图的一个未匹配点出发，依次交替经过未匹配边->匹配边->未匹配边->匹配边->...，最后以一个未匹配点结束**的路径，被称为增广路径。  
可以发现，增广路径上的未匹配边要比匹配边多一条，且路径上前后两条边之间存在公共顶点，因此**交换增广路径上的所有未匹配边和匹配边**，就可以使二分图中的匹配边数量加一。  
*注：如果我们基于当前的匹配，无法在二分图中找到增广路径时，那么可以确定当前的匹配即为二分图的最大匹配。*

## 数据结构设计

+ 使用邻接矩阵来表示二分图的拓扑关系。
+ 使用两个数组 $left[], right[]$ 来表示匹配边，那么对增广路径取反则简化为向数组 $left[]$ 或 $right[]$ 中添加顶点。

## 算法实现

在了解了增广路径的定义以后，我们其实可以大概猜测出匈牙利算法的流程，那就是不断的重复寻找增广路径、交换匹配边和未匹配边这一过程，直到无法找到增广路径。但是算法设计起来还是有一些复杂的，我们需要一些数据结构来标记匹配边、未匹配点、增广路径轨迹等。

### 搜索增广路径

+ 采用深度优先搜索的方式寻找增广路径，对每个顶点，则依次从其邻居进行查找，看能不能成功找到一个条增广路径。
+ 寻找增广路是一个递归的过程，为了避免重复搜索，需要使用数组 $used[]$ 标记每个顶点是否被探索过。
+ 最终的结果需要标记每个顶点的匹配点，使用 $matched[i]$ 来表示顶点 $i$ 是的对应匹配点，默认为 -1。

### 时间复杂度

+ 假设二分图的左侧顶点个数为 $m$，右侧顶点个数为 $n$ 。
+ 对二分图左侧的每个顶点进行一次搜索，每次搜索最多探索 $O(m*n)$ 条边，那么匈牙利算法的时间复杂度为 $O(m^2*n)$，如果 $m$ 和 $n$ 相等的话，复杂度就简化为 $O(n^3)$。

### 空间复杂度

+ 需要 $O(m+n)$ 大小空间的辅助数据来保存解和中间变量。

In [ ]:
# 标记匹配结果
def CallMarkNodes():
    for n in left_nodes:
        if hungaryMatch[n]:
            graph.markNode(algviz.colors[0], graph_map[n], False)
            graph.markNode(algviz.colors[1], graph_map[hungaryMatch[n]], False)

# 匈牙利算法实现。
viz._wait = True

hungaryMatch = viz.createVector([None for _ in range(len(graph_nodes))], name="MatchResult")
hungaryUsed = viz.createVector([False for _ in range(len(graph_nodes))], name="Used")

def hungaryFindPath(node):
    for neighbor in node.neighbors():
        CallMarkNodes()
        viz.display()
        if not hungaryUsed[neighbor[0].val]:
            hungaryUsed[neighbor[0].val] = True
            Temp = hungaryMatch[neighbor[0].val]
            if Temp == None or hungaryFindPath(graph_map[Temp]):
                hungaryMatch[node.val] = neighbor[0].val
                hungaryMatch[neighbor[0].val] = node.val
                CallMarkNodes()
                viz.display()
                return True
    return False

def hungarySolver():
    for n in left_nodes:
        for i in range(len(graph_nodes)):
            hungaryUsed[i] = False
        hungaryFindPath(graph_map[n])
        
hungarySolver()

# 参考链接

+ https://blog.sengxian.com/algorithms/km
+ https://zhuanlan.zhihu.com/p/89380238
+ https://blog.csdn.net/Young__Fan/article/details/90719285